In [1]:
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt
import glob

import matplotlib

from astropy.coordinates import SkyCoord
from astropy import units as u

%matplotlib inline

In [2]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [ ]:
selectnum = int(np.round(400000/26., ))
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees
for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     print(df.loc[df.SELECTED == True].shape[0])
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

In [17]:
selected = df[df.SELECTED == True]

In [18]:
selected.to_csv('../data/selected5.csv.bz2',compression='bz2')

# ok let's now try to select things kinda realistically

In [ ]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [ ]:
# have 6000 targets a each of the poles

selectnum = 8000
selectpoles = 6000
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees

#poles first
mn = (df.ECLAT >= 78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

mn = (df.ECLAT <= -78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6) & (df.ECLAT > -78)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6) & (df.ECLAT < 78)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

In [ ]:
# now add tvguide cameras
df['tess_ccd'] = np.zeros(df.shape[0], dtype='int')
df['obs_len'] = np.zeros(df.shape[0], dtype='float') 
for i in tqdm(df.index):
    #need a hack to force stars in the northern hemisphere to work
    if df.loc[i,'ECLAT'] >= 6:
        gc = SkyCoord(lon=df.loc[i,'ECLONG'] * u.degree, lat=df.loc[i, 'ECLAT'] * u.degree * -1,
                      frame='barycentrictrueecliptic')
        obj = TessPointing(gc.icrs.ra.value, gc.icrs.dec.value)
    else:
        obj = TessPointing(df.loc[i,'RA'], df.loc[i,'DEC'])
    df.loc[i,'tess_ccd'] = obj.get_camera()
    df.loc[i,'obs_len'] = obj.get_13cameras()[obj.get_13cameras() > 0].shape[0] * 27.4

In [ ]:
selected = df[df.SELECTED == True]

In [6]:
selected.to_csv('../data/selectedreal5.csv.bz2',compression='bz2')

In [11]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [13]:
gc = SkyCoord(ra=150*u.degree, dec=45*u.degree, frame='icrs')

In [20]:
gc.barycentrictrueecliptic.lon.value

135.31169020376774

# now do our alternative with 12,000 at the poles and 3,000 in each sector

In [ ]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [3]:


selectnum = 2000
selectpoles = 12000
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees

#poles first
mn = (df.ECLAT >= 78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

mn = (df.ECLAT <= -78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6) & (df.ECLAT > -78)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6) & (df.ECLAT < 78)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

0.0 27.6923076923 28000
27.6923076923 55.3846153846 32000
55.3846153846 83.0769230769 36000
83.0769230769 110.769230769 40000
110.769230769 138.461538462 44000
138.461538462 166.153846154 48000
166.153846154 193.846153846 52000
193.846153846 221.538461538 56000
221.538461538 249.230769231 60000
249.230769231 276.923076923 64000
276.923076923 304.615384615 68000
304.615384615 332.307692308 72000
332.307692308 360.0 76000


In [4]:
selected = df[df.SELECTED == True]

In [5]:
selected.to_csv('../data/selectedreal5-LongerBaseline.csv.bz2',compression='bz2')